# Vergabe NRW

3. Abfragen
Die Schnittstelle erlaubt sowohl GET Abfragen als auch Post Abfragen. Wir empfehlen POST Abfragen und gehen in diesem Dokument auf diese ein.

Die Schnittstelle wird über folgende URL angesprochen:
https://daten.vergabe.nrw.de/rest/vmp_rheinland_single/aggregation_search

Es ist keine Authentifizierung notwendig, da es sich um eine offene Schnittstelle handelt.
Wichtig ist, dass ein Header übergeben wird. Mindestens ist dieser ein Accept header:
Key: Accept
Value: application/json

Abbildung zeigt eine Post Abfrage mit dem Tool Postman und den passenden Header
Im Folgenden werden verschiedene Beispielabfragen beschrieben.

In [38]:
import requests 
import pandas as pd
import numpy as np
import json
import janitor
from tabulate import tabulate

#show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

np.set_printoptions(threshold=np.inf)

In [2]:
url = 'https://daten.vergabe.nrw.de/rest/vmp_rheinland_single/aggregation_search/'
 
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

params = {'size': 10000}
 
req = requests.post(url, headers=headers, params=params)
req

<Response [200]>

In [3]:
json_data = json.loads(req.text)
#json_data
df = pd.json_normalize(json_data['results'])
nomes = df.columns
nomes = nomes.str.replace('result\.', '')
df.columns = nomes
df = df.iloc[:,1:19]
df = df.clean_names()

# Only checking the data if imported ok
df.head(5)

,created_at,updated_at,id,procedure_type,order_type,publication_date,cpv_codes,buyer_name,buyer_address,buyer_town,buyer_postal_code,buyer_country,seller_name,seller_town,seller_country,geo_longitude,geo_latitude,value
0,2019-11-19T07:21:06.290Z,2019-11-19T07:21:06.290Z,CXPTYD0Y7U6,,,2019-11-19,[45000000],"Stadt Mönchengladbach, Dezernat Planen, Bauen,...",Markt 11,Mönchengladbach,41236,DE,,,,6.4459,51.16741,
1,2019-11-19T07:25:02.866Z,2019-11-19T07:25:02.866Z,CXPTYD4YQG4,Ex post Veröffentlichung (§ 20 Abs.3),VOB,,"[45410000-4, 45262500-6]",ENNI Stadt & Service Niederrhein AöR,Am Jostenhof 7-9,Moers,47441,NaN,,,,6.6084,51.45208,
2,2019-11-19T07:29:18.628Z,2019-11-19T07:29:18.628Z,CXPTYYFY7ZT,Öffentliche Ausschreibung,VOB,2019-11-19 9:00,[45112723-9],"Bundesstadt Bonn, Amt für Stadtgrün",Berliner Platz 2,Bonn,53111,NaN,,,,7.0944,50.73657,
3,2019-11-19T07:35:21.187Z,2019-11-19T07:35:21.187Z,CXPTYY5Y7VX,Ex ante Veröffentlichung (Binnenmarktrelevanz),VOB,,[45112720-8],"Stadt Ratingen, Der Bürgermeister, Hauptamt, Z...",Postfach 10 17 40,Ratingen,40837,NaN,,,,6.82157,51.29856,
4,2019-11-19T07:43:49.475Z,2019-11-19T07:43:49.475Z,CXS0Y6LYYMY,Ex ante Veröffentlichung (§ 19 Abs. 5),VOB,,[45300000-0],Stadt Langenfeld Rhld. - ZVS,Konrad-Adenauer-Platz 1,Langenfeld,40764,NaN,,,,6.95124,51.11277,


In [4]:
print('GENERAL INFORMATION: \n\n rows and columns number:', df.shape, 
     '\n older creation data:', df.created_at.min(),
     '\n newer creation data:', df.created_at.max())

GENERAL INFORMATION: 

 rows and columns number: (5817, 18) 
 older creation data: 2019-11-19T07:19:15.448Z 
 newer creation data: 2021-04-03T22:19:51.775Z


As said in the documentation, the maximal consult is 10.000 but the final document has only 5817 entries, englobing mid november 2019 until beggining of april 2021 (current month)

### Looking at null values:

In [5]:
nv = pd.DataFrame(df.isnull().sum())
nv.columns = [ 'total_null_values']
nv

,total_null_values
created_at,0
updated_at,0
id,1355
procedure_type,1355
order_type,1355
publication_date,1355
cpv_codes,1355
buyer_name,1355
buyer_address,1355
buyer_town,1355


### Checking null values:

In [6]:
df.query("id != id")

,created_at,updated_at,id,procedure_type,order_type,publication_date,cpv_codes,buyer_name,buyer_address,buyer_town,buyer_postal_code,buyer_country,seller_name,seller_town,seller_country,geo_longitude,geo_latitude,value
605,2019-11-19T07:43:34.461Z,2020-01-14T23:02:06.807Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
611,2019-11-19T07:19:40.608Z,2020-01-14T23:03:37.808Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
612,2020-01-05T00:45:51.687Z,2020-01-14T23:06:04.728Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
613,2020-01-05T00:07:41.743Z,2020-01-14T23:05:36.249Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
618,2020-01-07T23:05:51.547Z,2020-01-14T23:07:20.187Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
619,2020-01-09T09:08:22.294Z,2020-01-14T23:06:45.808Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
620,2019-11-19T07:22:32.727Z,2020-01-14T23:10:11.907Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
621,2019-11-19T07:22:42.089Z,2020-01-14T23:11:31.371Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
622,2019-11-19T07:22:54.771Z,2020-01-14T23:12:13.219Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
623,2020-01-04T23:07:40.708Z,2020-01-14T23:09:55.388Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1335 rows in the API are completelly empty 
<br> <br>
### Checking Cities

In [7]:
cities = pd.DataFrame(df.groupby('buyer_town')['buyer_town'].count())
cities.loc['Total']= cities.sum(numeric_only=True, axis=0)
cities.columns = ['vergabe_available']
cities

,vergabe_available
buyer_town,
Bad Honnef,27
Bergheim,285
Bergisch Gladbach,197
Bergisch-Gladbach,1
Bergneustadt,4
Bergsich Gladbach,1
Bonn,626
Bornheim,21
Brühl,116


Only 60 from the 396 cities from NRW are available (15%).
<br>
### Order type:

In [8]:
ot = pd.DataFrame(df.groupby('order_type')['order_type'].count())
ot.loc['Total']= ot.sum(numeric_only=True, axis=0)
ot.columns = ['vergabe_available']
ot

,vergabe_available
order_type,
,140
OTHER,30
UVGO,1392
VOB,2896
VOL,4
Total,4462


### Distribution of publication_date

Trying to see if any month could be missing in our database:

In [9]:
ca = df
ca['created_at'] = pd.to_datetime(ca['created_at']) 
ca['year'] = ca['created_at'].dt.year
ca['month'] = ca['created_at'].dt.month
ca2 = pd.DataFrame(ca.groupby(['year','month'])['id'].count())
ca2

id
year month     
2019 11     667
     12     190
2020 1      128
     3        0
     4      616
     5      245
     6      448
     7      369
     8      261
     9        8
     11     115
     12     313
2021 1      406
     2      273
     3      400
     4       23

### CPV CODE
Now we are going to look at CPV codes to see if we can find a pattern

In [10]:
cpvcode = pd.read_csv('cpvcode.csv').clean_names()

#Checking importing
cpvcode.head(2)

#Checking if every valu in column code contain just one cpv code:
cpvcode['code_count'] = cpvcode['code'].str.len()
cpvcode.groupby(['code_count'])['code_count'].count()  # all have just one CPV code

#Droping column
cpvcode = cpvcode.drop(columns=['code_count'])

cpvcode.shape # (9454, 4)

cpvcode.head()

,code,de,en,pt
0,03000000-1,Landwirtschaftliche Erzeugnisse des Pflanzenba...,"Agricultural, farming, fishing, forestry and r...","Produtos da agricultura, da pesca, da silvicul..."
1,03100000-2,Landwirtschafts- und Gartenbauerzeugnisse,Agricultural and horticultural products,Produtos agrícolas e hortofrutícolas
2,03110000-5,Feldfrüchte und Erzeugnisse des Erwerbsgartenbaus,"Crops, products of market gardening and hortic...","Cereais, produtos de culturas industriais e da..."
3,03111000-2,Saatgut,Seeds,Sementes
4,03111100-3,Sojabohnen,Soya beans,Soja


In [11]:
#checking types of columns :

print('cpvcode:',cpvcode.dtypes,'\n\n\n df:', df.dtypes)

cpvcode: code    object
de      object
en      object
pt      object
dtype: object 


 df: created_at           datetime64[ns, UTC]
updated_at                        object
id                                object
procedure_type                    object
order_type                        object
publication_date                  object
cpv_codes                         object
buyer_name                        object
buyer_address                     object
buyer_town                        object
buyer_postal_code                 object
buyer_country                     object
seller_name                       object
seller_town                       object
seller_country                    object
geo_longitude                     object
geo_latitude                      object
value                             object
year                               int64
month                              int64
dtype: object


In [46]:
# In our DF, some rows have more than one cpv code. Lets see how many:

df_cpv = df
df_cpv['cpv_codes2'] = df_cpv['cpv_codes'].astype(str)
df_cpv['code_count'] = df_cpv['cpv_codes2'].str.len()


df_cpv.groupby(['code_count'])['code_count'].count()  # 1 row has 406 characteres

x = df_cpv.query('code_count == 406')
x.cpv_codes2 # 597

#criando 597 colunas

n = []

for i in range(0,600):
    i = str(i)
    x = 'CPV' + i
    n.append(x)

df_cpv['cpv_codes2']  = df_cpv['cpv_codes2'].str.replace('\[\'', '') 
df_cpv['cpv_codes2']  = df_cpv['cpv_codes2'].str.replace('\'\]', '') 

# Df has more than one CPV code, so we need to split it, when it happens.
# In our new dataframe, each row will became the vergabe for one cpv code, so the vergaben_id (df.id) will repeat 


df_cpv[[n]] = df["cpv_codes2"].str.split(pat=",", expand=True)
df_cpv.head(2)

ValueError: Columns must be same length as key

# Questions and assumptions:

Verificar:

- why is there buyer country? We believe that the rows which contain the buyer country were purchases that needed to be reported at EU level 
- every awarded conttact should have a seller information
- why are there rows without any procedure type?